In [ ]:
https://github.com/dathwal/DSCI100-Project.git

First, we load in the necessary libraries.

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(themis)


options(repr.matrix.max.rows = 6)


The two datasets (players.csv and sessions.csv) are then loaded in using the read_csv function. To help with 
visualizing the column names, the glimpse function is used. 

In [ ]:
players<- read_csv("players.csv")

glimpse(players)


In [ ]:
# loaded Data Set
sessions <- read_csv("sessions.csv")
glimpse(sessions)


The question that I wish to answer is predictive, so the method that we will use is classification. We wish to find clusters of data that will hopefully show if there is a particular age group that contributes the most played hours so that we are able to target this age group with advertising. First, tidy the data. 

In [ ]:
players_cleaned <- players |>
select(played_hours, experience)|>
mutate(experience = as_factor(experience))
players_cleaned

This cleaned data can now be used to create a plot to observe any relationships between the two variables.

In [ ]:
players_plot <- players_cleaned|>
ggplot(aes(x=played_hours, y=experience))+
geom_point() +
ggtitle("Played Hours vs Time")+
xlab("Played Hours")

players_plot

The dataset can now be split into training and testing sets.

In [ ]:
set.seed(1543)
players_split <- initial_split(players_cleaned, prop=0.7,strata=experience)
players_training <- training(players_split)
players_training
players_testing <- testing(players_split)

A recipe is then created based on the training set.

In [ ]:
k_recipe <- recipe(experience ~ played_hours, data = players_training)|>
step_scale(all_numeric())|>
step_center(all_numeric())|>
step_upsample(experience, over_ratio=1)

A KNN classification model and is then tuned and fitted.

In [ ]:
k_spec <- nearest_neighbor(weight= "rectangular", neighbors=9)|>
set_engine("kknn")|>
set_mode("classification")

The model and recipe are then fitted with the training data using a workflow.

In [ ]:

k_fit <- workflow()|>
add_recipe(k_recipe)|>
add_model(k_spec)|>
fit(data = players_training)

k_fit

The fitted model is then evaluated by predicting the classes in the testing set. 

In [ ]:
k_predict <- k_fit|>
predict(players_testing)|>
bind_cols(players_testing)|>
metrics(truth=experience, estimate = .pred_class)|>
filter(.metric== "accuracy")
k_predict

In [ ]:
players_graph <- players_training |>
ggplot(aes(y=experience, x=played_hours,colour=experience))+
geom_point()

players_graph

Individual Report
A research group at UBC collected data about players of a MineCraft server and aimed to use this infoamtion collected to learn more aboimprove advertising and 
The question that this project was centered around was to see if player experience influenced the amount of hours played in order to see if we can target advertising of this Minecraft server to a specific group of people. 